# Workshop 1 - Water #

## Introduction ##

Water is the most common and important chemical solvent - it is not only central to life, it plays a critical role in colloid, interface and soft matter systems. Solvents such as water are typically handled in MD simulations via distinct force fields called _**solvent models**_, designed specifically to describe a solvent environment around a solute (e.g. micelle, polymer, protein) as accurately and efficiently as possible. Solvent models constitute a distinct group of MD force fields – they consist of parameters that are optimized to reproduce specific experimental properties of that particular liquid, such as density, enthalpies of vaporization and self-diffusion coefficients. 

As with any MD simulation, to simulate liquid water we must provide information about each water molecule’s initial position and velocity, as well as the forces that water molecules exert on each other. However, solvent water models involve additional parameters, such as whether each water molecule is flexible, or held fixed (and if so, what the geometry of each water molecule is), how the charge is distributed within each water molecule, and how each water molecule interacts with those around it via van der Waals interactions. Each of these parameters greatly affect how physically realistic the water model’s characteristics are. Each different water model will therefore have its own successes and weaknesses. 


### MD Water Models ###

A perfect model of liquid water would use quantum mechanics to exactly solve the system's Schrödinger equation, an impossible task, even on modern computers. Instead, assumptions are made to create water ‘models’ designed for different purposes to speed things up. Water models generally fix the geometry of individual molecules and replace their H and O atoms with point charges and describe van der Waal interactions via a simple Lennard-Jones potential. In other words, the energy of water is described by, 

$$E_{ab} = \sum_{i}^{i \in \text{mol. A}}\sum_{j}^{j \in \text{mol. B}}\frac{k_C q_i q_j}{r_ij} + \frac{A}{r_{OO}^{12}} - \frac{B}{r_{OO}^{6}}$$

Notice that this expression does not depend on the distance between H and O atoms in the **same** water molecule. This means that the geometry in each water molecule is fixed. The total interaction between pairs of water molecules using this model therefore arises from 3 sites, and is just the sum of their electrostatic interaction (the first term), and their van der Waals interactions (the 2nd and 3rd terms). The latter is assumed to arise only from the oxygen atom. 

Simulating liquid water with 3-site water models therefore only requires **4 parameters** ($q_{O}$, $q_{H}$, $A$ and $B$) - this makes MD calculations extremely efficient. However, since there are only 4 parameters, water models are notoriously inflexible. For instance, values of $q_{O}$, $q_{H}$, $A$ and $B$ that give an accurate prediction of one property, such as water density, may give an inaccurate prediction of the heat of vaporisation. In the Table below, TIP3P and SPC/E are both examples of 3-site water models - we will be using TIP3P in this workhsop. 

One of the most popular attempts to solve this problem is to introduce "fake" atoms into the water molecule structure. These fictitious **dummy atoms** ("M") atoms have no mass, but a non-zero charge. They are generally placed near the oxygen atom, effectively changing the water molecule's dipole moment - and therefore the  electrostatic and van der Waals interactions between pairs of water molecules. Effectively the presence of the dummy atom in this water model mimics, in a crude way, the presence of the oxygen lone electron pairs. Such "4-site" water models rely on **5 parameters** ($q_{O}$, $q_{H}$,$q_{M}$, $A$ and $B$), so have an increased degree of flexibility and can predict some properties of liquid water more accurately. However this comes at the cost of (1) having to determine what the charge on the dummy atom M is, and where it should reside in the water molecule (see **r(OM)** in the table below). More importantly, the inclusion of an additional atom in the force field means that MD simulations will be slightly less efficient. In the Table below, TIP4P-Ew, TIP4P/Ice, TIP4P/2005 and OPC are all examples of 4-site water models, each designed to predict some specific property of water (for instance, the parameters of the TIP4P/Ice model were specifically designed for simulating different phases of solid water). In this workshop we will examine the performance of TIP4P-Ew.

Yet more advanced water models have also been developed, such as the 5- and 6-site water models. An example of the 5-site water model is shown below - instead of a single dummy atom M, 5-site water models incorporate a pair of dummy atoms ("L") that mimic more accurately the position of the oxygen lone electron pairs. 6-site models incorporate these lone pair dummy atoms and the 4-site dummy atom M together. The incorporation of multiple dummy atoms generally improves the prediction of these models, but with a significant increase in model complexity and simulation  efficiency.





<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Water_models.svg/1024px-Water_models.svg.png" align="center"/>

**Examples of 3- & 4-Site Water Models**
| Model | $q_{O}$ (e) | $q_{H}$ (e) | $q_{M}$ (e) | r(OH) ($\mathring A$) | $\theta_{\text{HOH}}$ (deg.) |  r(OM) ($\mathring A$)  | A ($10^{3}$kcal$\mathring A^{12}$mol$^{-1}$)  | B  ($10^{3}$kcal$\mathring A^{12}$mol$^{-1}$) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| TIP3P | -0.834 | +0.417| -  | 0.9572 | 104.52 | - | 582.0 | 595.0 | 
| SPC/E | -0.8476 | +0.4238 | - | 1.0 | 109.47 | - | 629.4 | 625.5 |
| TIP4P-Ew | - | +0.52422 | −1.04844 | 0.9572 | 104.52 | 0.125 | 656.1  | 653.5  |
| TIP4P/Ice | - | +0.5897 | −1.1794 | 0.9572 | 104.52 | 0.1577 | 857.9  | 850.5 |
| TIP4P/2005 | - | +0.5564 | −1.1128 | 0.9572 | 104.52 | 0.1577 | 731.3  | 736.0 |
| OPC | | +0.6791 |  −1.3582 | 0.8724 | 103.6 | 0.1546 | 865.1 | 858.1 |
| **Exp. water (gas-phase)**  |- |- | - | **0.9572** | **104.52** | - | - |

### Workshop Aims ###

In this workshop, we examine how well 3- and 4-site water models reproduce structural and dynamic properties of water under various physical conditions.

The workshop addresses three questions:
 
1. How reliable are the TIP3P and TIP4P-Ew  water models in reproducing the experimental density for liquid water at 298 K and 1 atm?
1. How reliable are the TIP3P and TIP4P-Ew  water models in reproducing the structure of water at 298 K and 1 atm?
1. How reliable are the TIP3P and TIP4P-Ew  water models in reproducing the self-diffusion dynamics of water at 298 K and 1 atm?


## Simulating Bulk Water ##

### TIP3P Water Simulation ###

#### Setting Things Up  ###

First things first - as we saw in the previous introductory exercise, several modules need to be first loaded into our Jupyter notebook so that we can run our MD simulation. Let's take care of this now:

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import re
import time
import numpy as np

Now let's define the physical parameters of our simulation. You can use the panels below to change them, however for now it is ok to leave them at their present values. 

In [ ]:
import ipywidgets as widgets 
from IPython.display import display

#SIMULATION TEMPERATURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=298.0,  # default value 
    description='Simulation Temperature (K):',
    step=1.0,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTemperature
    simTemperature = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTemperature = temp_text.value*kelvin

#BOXSIZE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=24.0,  # default value 
    description='Box Size (Angstrom):',
    step=1.0,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global boxsize
    boxsize = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
boxsize = temp_text.value

#TIMESTEP
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1.0,  # default value 
    description='Time step (fs):',
    step=0.5,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTimestep
    simTimestep = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTimestep = temp_text.value*femtoseconds

#PRESSURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1.0,  # default value 
    description='Simulation pressure (atm):',
    step=0.10,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simPressure
    simPressure = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simPressure = temp_text.value*atmospheres

#STEPS
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=100000,  # default value 
    description='# MD steps:',
    step=100,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simNumSteps
    simNumSteps = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simNumSteps = temp_text.value

The parameters above do not specify the the # of waters we will place in our simulation box. But knowing the size of the box, and the density of water (0.9987 g/cm$^3$ at STP) we can work out how many waters we will have:

In [ ]:
density=0.99705
water=int(boxsize**3*density*1e6*(1e-10)**3/(18.01)*6.02214e23)
print("Box size of",boxsize,"angstroms at density,",density,"g/cm3 requires ",water," water molecules")

For now, you can leave these parameters unchanged from their default values. Let's make sure their stored correctly by executing the next cell:

In [ ]:
print(f"Simulation size: {water} waters")
print(f"Box size: {boxsize} Ang.")
print(f"Simulation temperature: {simTemperature}")
print(f"Simulation pressure: {simPressure}")
print(f"Simulation time step: {simTimestep}")
print(f"# MD steps: {simNumSteps} ; Total simulation length: {simNumSteps * simTimestep} = {simNumSteps*simTimestep / 1000 / 1000 / femtoseconds * nanoseconds}.")
pdbFile = 'system.pdb'

#### Generating the Unit Cell ####

With the parameters for our simulation now defined, we need to generate the actual "box" of water molecules that we will use for the simulation. We will use an external program call ```packmol``` to do this. ```packmol``` is widely used for generating condensed phase systems in biology, materials science and colloid & interface science. Here we are only generating a water box, but it is capable of [much more complicated stuff](https://m3g.github.io/packmol/examples.shtml) as well. 

To start with, we will prepare the input file, ```packmol.in```, that ```packmol``` will go looking for. This input file tells ```packmol``` to build a box of waters according to the parameters that you just set above:

In [ ]:
with open("packmol.in", "w") as f:
    f.write("tolerance 2.0\n")
    f.write("filetype pdb\n")
    f.write("output system.pdb\n")
    f.write("\n")
    f.write("structure water.pdb\n")
    f.write("  number {0}\n".format(int(water)))
    f.write("  inside cube 2. 2. 2. {0}\n".format(boxsize))
    f.write("end structure\n")
    f.write("\n")
    f.write("add_box_sides 1.0\n")


Now we run ```packmol``` itself, and make sure that everything worked ok:

In [ ]:
#%%bash
!/home/chem3580/Downloads/ENTER/envs/openMM/bin/packmol < packmol.in > packmol.out

In [ ]:
with open('packmol.out', 'r') as f:
    text = f.read()
    print(text)

Let's take a look at the simulation box that you've just generated: 

In [ ]:
with open('system.pdb', 'r') as f:
    text = f.read()
    print(text)

Notice here that the only particles in the simulation box so far are hydrogen and oxygen atoms. This is fine for the TIP3P water model, however a bit later we will have to adjust this file to include the "M" dummy atoms when we use the TIP4P water model. 

#### Defining the MD Simulation ####

We are now ready to setup the MD simulation objects using the ```OpenMM``` python module. The process for this is _exactly_ the same as that in the introductory workshop - we need to define: 

1. What forcefield we are going to use (this defines how bonded and non-bonded atoms interact with each other)
1. What the system topology is (i.e. which atoms are connected to which atoms etc.)
1. How we will control the simulation temperature and pressure (e.g. using a _thermostat_ or a _barostat_), and
1. How we will iterate Newton's equations of motion (i.e. the _integrator_)

Run the cells below. They will setup the simulation in the following way: 

1. We will apply the TIP3P water model forcefield and store it in  the ```tip3p_forcefield``` object. 
1. We combine the ```tip3p_forcefield``` object with the topology in our PDB file ```system.pdb``` (generated above using packmol) to define the MD simulation, which we store in the ```tip3p_system``` object. 
1. We add a barostat (the ```MonteCarloBarostat```) to the simulation using ```OpenMM```'s ```addForce``` module. 
1. We use ```OpenMM```'s ```NoseHooverIntegrator``` to iterate the equations of motion.


In [ ]:
pdb = PDBFile('system.pdb')

In [ ]:
tip3p_forcefield = ForceField('tip3p.xml')

In [ ]:
tip3p_system = tip3p_forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=11*angstrom,
    constraints=None,
    rigidWater=False)

In [ ]:
tip3p_system.addForce(MonteCarloBarostat(simPressure, simTemperature, 1))

In [ ]:
tip3p_integrator = NoseHooverIntegrator(
    simTemperature, 
    1.0/picosecond, 
    simTimestep
)

Finally, we tie all of these details together in the ```tip3p_simulation``` object, via ```OpenMM```'s ```Simulation``` module:

In [ ]:
tip3p_simulation = Simulation(pdb.topology, tip3p_system, tip3p_integrator)
tip3p_simulation.context.setPositions(pdb.positions)

Now we are ready to run!! 

First things first - we have to minimise the energy of our water box to a local minimum on the potential energy surface. Run the next cell to do this: 

In [ ]:
print("Running Energy Minimisation:")
t0 = time.time()
tip3p_simulation.minimizeEnergy()
t1 = time.time()
minTime = t1-t0
print("Minimisation took {} seconds".format(minTime))

The arrangement of waters in your box should now correspond to a local energy minimisation, meaning that you are good to go - run the MD simulation!

Before we do though, we want to make sure that we store the relevant data from the MD trajectory. The cell below will take care of this for you, and will track the progress of the MD simulation in a file ```tip3p_Equil.dcd```:

In [ ]:
tip3p_simulation.reporters.append(DCDReporter('tip3p_water.dcd', 10))
tip3p_simulation.reporters.append(StateDataReporter(
    'tip3p_water.csv', 
    10, 
    step=True,
    temperature=True, 
    potentialEnergy=True, 
    kineticEnergy=True,
    totalEnergy=True, 
    volume=True, 
    density=True
))

Run the next cell to run your MD simulation!

In [ ]:
print('Start Simulation')
t0 = time.time()
tip3p_simulation.step(simNumSteps)
t1 = time.time()
simTime = t1-t0
print("Simulation took {} seconds for {} timesteps".format(simTime,simNumSteps)) 

### TIP4P-Ew Water Simulation ###

We'll now re-simulate the same box of water using a different force field for a comparison. The setup is a little simpler here, since we do not have to load python modules, construct the simulation box with ```packmol```, define the MD simulation parameters etc. 

We only need to setup the new MD simulation object using ```OpenMM```. The process for this is _exactly_ the same as that above - recall, we need to define: 

1. What forcefield we are going to use (this defines how bonded and non-bonded atoms interact with each other)
1. What the system topology is (i.e. which atoms are connected to which atoms etc.)
1. How we will control the simulation temperature and pressure (e.g. using a _thermostat_ or a _barostat_), and
1. How we will iterate Newton's equations of motion (i.e. the _integrator_)

Run the cells below. They will setup the simulation in the following way: 

1. We will apply the TIP4P-Ew water model forcefield and store it in  the ```tip4pew_forcefield``` object. 
1. We combine the ```tip4pew_forcefield``` object with the topology in our PDB file ```system.pdb``` (generated above using packmol) to define the MD simulation, which we store in the ```tip4pew_system``` object. 
1. We add a barostat (the ```MonteCarloBarostat```) to the simulation using ```OpenMM```'s ```addForce``` module. 
1. We use ```OpenMM```'s ```NoseHooverIntegrator``` to iterate the equations of motion.

Notice that we are only changing **1 aspect** of our MD simulation - the water model itself - _all other aspects of the MD simulation are the same._ This means that any & all differences between our TIP3P and TIP4P-Ew simulations can only be caused by the differences in the water models themselves. 

In [ ]:
tip4pew_forcefield = ForceField('tip4pew.xml')

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addExtraParticles(tip4pew_forcefield)
PDBFile.writeFile(modeller.topology, modeller.positions, open('system_tip4pew.pdb', 'w'))
tip4pew_pdb = PDBFile('system_tip4pew.pdb','M')

In [ ]:
tip4pew_system = tip4pew_forcefield.createSystem(tip4pew_pdb.topology,nonbondedMethod=PME,nonbondedCutoff=11*angstrom,constraints=None,rigidWater=False)
tip4pew_system.addForce(MonteCarloBarostat(simPressure, simTemperature, 1))
tip4pew_integrator = NoseHooverIntegrator(
    simTemperature, 
    1.0/picosecond, 
    simTimestep
)
tip4pew_simulation = Simulation(tip4pew_pdb.topology, tip4pew_system, tip4pew_integrator)
tip4pew_simulation.context.setPositions(tip4pew_pdb.positions)

You now have your TIP4P-Ew water MD simulation set up and ready to run. Now it's your turn to write some of your own code!

#### Exercise ####

1. Copy and paste the **energy minimisation** python code above from your TIP3P simulation, edit your new code as needed, and minimise the energy of your water simulation box using the TIP4P water model. _NOTE - remember to change 'tip3p' to 'tip4pew' in all variable names!_
1. Copy and paste the **MD simulation** python code above from your TIP3P simulation, edit your new code as needed, and run the MD simulation of your water simulation box using the TIP4P water model. _NOTE - remember to tell your Jupyter notebook to record your trajectory in a DCD file first!!_

##### Important Notes #####
1. Insert as many code cells in the jupyter notebook below as you need. Try and keep your code as 'clean' as possible - it will help you solve and debug problems if and when they happen!
1. If you get stuck, ask your demonstrator for help!
1. <span style="color:red;">REALLY IMPORTANT - make sure that you change all "tip3p" to "tip4pew" in your new code below. Otherwise you will destroy the TIP3P water simulation you just ran!</span>

## Analysis - Have we Reached Equilibrium?? ##

With our 2 water MD simulations now complete, the first thing we have to ensure is that we have equilibrated both systems. As we have discussed in the lectures, the thermodynamic variables that we expect to be equilibrated (i.e. be, on average, constant in the equilibrated simulation) depends on the _ensemble_ that we are enforcing on the system. Here we are using an **NPT ensemble**. This means we need to check that both the temperature and pressure have equilibrated. We will measure the latter via the system density (since a liquid's density is determined by the pressure imposed on it - if density is constant, then pressure must also have equilibrated). 

All of the information we need to check equilibration was stored in the ```tip3p_water.csv``` file above for the TIP3P simulation, and the file you specified for the TIP4P-ew simulation. Let's quickly take a look at the contents of these files: 

In [ ]:
with open('tip3p_water.csv', 'r') as f:
    text = f.read()
    print(text)

This is a ```csv``` file (comma separated values), which can be easily imported into programs like MS Excel. It is just as easy though to quickly plot the data in this file using Python's ```Numpy``` module using the ```genfromtxt``` function. We will do this below and store the contents of this file in the variable ```data```:

In [ ]:
data = np.genfromtxt('tip3p_water.csv', skip_header=1,delimiter=',')

The variable ```data``` is a 2D array, where individual columns correspond to the columns in the file ```tip3p_water.csv```. The first column is the # of MD time steps in the MD simulation. In Python, this is not the 1st column in the ```data``` array, it is actually the 0th column. Let's use this column to define the array ```time``` which contains the physical times in picoseconds of these MD time steps: 

In [ ]:
timestep=0.001 #units of ps
time = data[:,0]*timestep

We now want to store the temperature and density data columns in ```data``` - above we can see that these are the 4th and 6th columns, respectively:

In [ ]:
temperature = data[:,4]
density = data[:,6]

With the time, temperature and density data now saved, we can use the ```matplotlib``` python module, as we have before, to show how the simulation temperature and density is changing over time: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(time,density)
plt.xlabel("Time (ps)")
plt.ylabel("Density (g/cm$^3$)")
plt.show()

In [ ]:
plt.plot(time,temperature)
plt.xlabel("Time (ps)")
plt.ylabel("Temperature (K)")
plt.show()

What are these two plots telling us? Firstly, looking at the evolution of simulation temperature vs time, we see that the initial temperature of the simulation is exactly 0 K - this is because our energy minimisation did not take into account thermal vibrations/motions of the molecules in the simulation. Once we start running the NPT-MD simulation itself, the temperature - very quickly - rises to ~298 K, which is the target temperature for ensemble. This is the effect of the **MD thermostat** that we have applied to the ensemble. In effect, the thermostat is forcing the motion of the atoms in the system to accelerate until the velocities of their velocities on average correspond to a constant value of 298 K. 

While this is happening, the density of the liquid gradually equilibrates to ~1 g/cm$^3$ - this is the effect of the **MD barostat** that we have applied to the ensemble. The barostat is not actually changing the density, rather it is changing the volume of the box (which depends on the surrounding pressure), but since the number of molecules in the simulation is constant, the density changes when the box volume changes. 

Have we equilibrated the simulation? Since we have an NPT ensemble, we need to consider if N, P and T are all approximately conserved within the timescale of the simulation:

1. N - this is trivial, since it the number of molecules in the box are not chaning. 
1. P - Pressure varies quite significantly, but within the last ~20 ps of the simulation averages to a relatively constant value (which we will calculate in a moment). 
1. T - Temperature appears to be oscillating closely around our target temperature of 298 K within the first ~20 ps of the simulation, so this is conserved as well. 

What are the values of our simulation temperature and density? We can use Python's ```Numpy``` module to calculate these values quite easily using the code below. 

<span style="color:red;">NOTE - the code below calculates the average density between 80-100 ps, and the average temperature between 20-100 ps. These ranges are appropriate based on the data above, but YOUR simulation may require you to specify different ranges in the code below.</span>



In [ ]:
average_density=np.mean(density[8000:10000]) 
stdev_density=np.std(density[8000:10000])
print("Density is",average_density,"+/-",stdev_density," g/cm3")
average_temperature=np.mean(temperature[2000:10000])
stdev_temperature=np.std(temperature[2000:10000])
print("Temperature is",average_temperature,"+/-",stdev_temperature," K")

For this simulation, it appears that we have a converged NPT ensemble! This means that we can use it to reliably measure structural and dynamic properties of liquid water. 

In [ ]:
data = np.genfromtxt('tip4pew_water.csv', skip_header=1,delimiter=',')
timestep=0.001 #units of ps
time = data[:,0]*timestep
density = data[:,6]
temperature = data[:,4]

In [ ]:
plt.plot(time,density)
plt.xlabel("Time (ps)")
plt.ylabel("Density (g/cm$^3$)")
plt.show()

In [ ]:
plt.plot(time,temperature)
plt.xlabel("Time (ps)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
average_density=np.mean(density[8000:10000])
stdev_density=np.std(density[8000:10000])
print("Density is",average_density,"+/-",stdev_density," g/cm3")
average_temperature=np.mean(temperature[2000:10000])
stdev_temperature=np.std(temperature[2000:10000])
print("Temperature is",average_temperature,"+/-",stdev_temperature," K")

### Exercise ###

1. Use the code provided above to repeat the density and temperature analysis for the TIP4Pew water model MD simulations that you ran previously. 
1. Compare your simulated water density and temperature with the TIP3P values calculated previously, and with the experimental data provided above. 

##### Important Notes #####
1. Insert as many code cells in the jupyter notebook below as you need. Try and keep your code as 'clean' as possible - it will help you solve and debug problems if and when they happen!
1. If you get stuck, ask your demonstrator for help!
1. <span style="color:red;">REALLY IMPORTANT - make sure that you change all "tip3p" to "tip4pew" in your new code below. Otherwise you will destroy the TIP3P RDFs you just calculated!</span>

## Analysis - Water Structure & the Radial Distribution Function ##

Having demonstrated that we have achieved an equilibrium NPT ensemble in the previous section, we can now see what this ensemble tells us about the structure of water itself. 

The two extreme scenarios of molecular interaction are solids, having molecules with strong interactions, which are aligned in a rigid crystal, and gases, having molecules with no to minimal interactions, with essentially no regular arrangement in their structure. The amount of interaction (and thus structure) of molecules in a liquid falls in the middle. This is one of the things that makes liquids so interesting - ***and the main reason almost all chemistry occurs in a solvent!***

The structure in a liquid can be characterized by a radial distribution function (RDF), g(r). An RDF is a measure of the radial frequency of particles around some point in space. Typically, we use RDFs to measure the likelihood of finding an atom type (e.g. oxygen) around another atom type (e.g. hydrogen). For a monatomic gas, g(r) is flat, because any atom is a random distance from all other atoms. There is no preferred distance; all are equally likely. For a crystalline solid, g(r) has regular, sharp peaks, because atoms are regularly spaced from each other - you will be familiar with this from X-ray diffraction patterns. In a solid, only some distances are likely; others are essentially forbidden. 

Liquids, on the other hand, are structured locally but random at long range. This means a liquid g(r) resembles that of a solid at short-range, but that of a gas at long-range. Most liquids typically exhibit a few distinct short range peaks, these correspond to **solvation shells**. The position of these peaks indicate the length scale of each solvation shell, and the height of that peak indicates how pronounced the corresponding shell is in structure, relative to the surrounding bulk liquid environment. Eventually, as the nonlocal density becomes identical to the bulk density, g(r) flattens and tends to the bulk value of 1. 

For water, the amount of local structure can be characterized via the O-O, O-H and H-H g(r) functions (i.e. "oxygens around oxygens", "hydrogens around oxygens" and "hyrogens around hydrogens"), which are shown in the Figure below. This figure is produced via neutron diffraction measurements, however MD is also capable of producing RDFs. These RDFs show that: 

1. The hydrogen bonding structure in bulk water (i.e. the O-H g(r))  extends well beyond the first solvation shell (there are two hydrogen bond interactions at ~2 and 3.5 $\mathring A$, respectively). 
1. The interaction length between neighbouring water molecules (via the O-O g(r)). The figure below shows that water arranges itself into ~3 distinct solvation shells at ~2.5, 4.5 and 7 $\mathring A$, respectively. 

So, water is **really** structured!! Now that we understand this, we can begin to understand the physical and chemical properties of water, and why some of them change in the presence of solutes and interfaces (as we will see in the next workshop). 

<img src="https://static-02.hindawi.com/articles/isrn/volume-2013/279463/figures/279463.fig.0021.jpg" align="left" width="500" height="500"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/e/ea/Molecular_Schematic_for_Interpreting_a_Radial_Distribution_Function.png" align="center" width="400" height="400"/>


Let's take our MD trajectory and use the ```MDAnalysis``` python module to calculate the same RDFs as shown in the Figure above. 

In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.rdf as rdf

MDAnalysis is extremely powerful, and relies on what is known as a 'universe' (essentially, all the information that defines an MD trajectory, i.e. the topology, the coordinates and the force field). Once we have defined a universe based on our trajectory, we can perform the RDF analysis. Let's store the universe in the variable ```tip3p_u```:

In [ ]:
tip3p_u = mda.Universe('system.pdb','tip3p_water.dcd') #NOTE - make sure you set this .pdb filename correctly for your TIP4Pew simulation below

We now need to tell ```MDAnalysis``` which pairs of atoms we want to investigate. To start with, let's look at the main O-O interaction and the O-H hydrogen bonding interaction in your bulk water MD simulation. We will use the ```select_atoms``` function to do this, and store the list of oxygen atoms in the variable ```O_tip3p```, and the list of hydrogen atoms in the variable ```H_tip3p```:

In [ ]:
O_tip3p = tip3p_u.select_atoms('name O')
H_tip3p = tip3p_u.select_atoms('name H1 or name H2')

Notice here that we are telling ```MDAnalysis``` specific atom labels (e.g. H1, H2, which are the two distinct H atoms in the water molecule), which match the labels in the PDB file produced previously by ```packmol```. 

Now we have to tell ```MDAnalysis``` the details of the RDF that we are after: 

In [ ]:
OO_rdf_tip3p = rdf.InterRDF(O_tip3p,O_tip3p,nbins=100,range=(1.0,10))
OH_rdf_tip3p = rdf.InterRDF(O_tip3p,H_tip3p,nbins=100,range=(1.0,10))

Here, we are using MDAnalysis's ```InterRDF``` function to calculate the RDF between each oxygen and every other oxygen, and store this in the ```OO_rdf``` object. The same information for the O-H RDF will be stored in the ```OH_rdf_tip3p``` object. For both RDFs, we will measure the RDF between 1 $\mathring A$ and 10 $\mathring A$ (see the ```range``` variable) at 100 points (see the ```nbins``` variable).

Now let's run the analysis:

In [ ]:
OO_rdf_tip3p.run()
OH_rdf_tip3p.run()

This should only take a minute or so. When it is finished, we can plot the results and look at the RDF! We will use the ```matplotlib``` python module, as we have before, to do this. Note that the horizontal and vertical axis data we want to plot are actually now in the ```rdf_calc``` object, as the ```rdf_calc.bins``` and ```rdf_calc.rdf``` variables, respectively. So we will pass these two variables to the plot command below:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(OO_rdf_tip3p.bins,OO_rdf_tip3p.rdf,label="O-O")
plt.plot(OH_rdf_tip3p.bins,OH_rdf_tip3p.rdf,label="O-H")
plt.xlabel("r ($\mathrm{\AA}$)")
plt.ylabel("g(r)")
plt.legend()
plt.ylim([0.0, 5])

#### Exercise ####

1. Use the code provided above to repeat the RDF analysis for the TIP4Pew water model MD simulations that you ran above. 
1. Use the code provided above to produce a single plot that compares the TIP3P and TIP4Pew water models' predictions for the O-O radial distribution functions
1. Use the code provided above to produce a single plot that compares the TIP3P and TIP4Pew water models' predictions for the O-H radial distribution functions
1. Compare your simulated water structure with the experimental data provided above. 

##### Important Notes #####
1. Insert as many code cells in the jupyter notebook below as you need. Try and keep your code as 'clean' as possible - it will help you solve and debug problems if and when they happen!
1. If you get stuck, ask your demonstrator for help!
1. <span style="color:red;">REALLY IMPORTANT - make sure that you change all "tip3p" to "tip4pew" in your new code below. Otherwise you will destroy the TIP3P RDFs you just calculated!</span>

## Analysis - Self Diffusion Coefficients in Bulk Water ##

Thermal fluctuations in a liquid cause a dissolved molecule to wander from its original position, executing a ‘random walk’. Eventually, the displacement of the molecule  is determined by its  **diffusion coefficient, D** (measured in area per time, e.g. m2/s). The diffusion coefficient is a proportionality constant, which relates the rate of diffusion of the molecule to that molecule's concentration gradient. Effectively, molecules and particles will move faster to a region where their concentration is lower (you know this already – drop some food colouring in water, and watch what happens!). 

The **self-diffusion coefficient**, as opposed to the diffusion coefficient, specifically refers to a molecule’s diffusion relative to its own bulk motion. That is, the self-diffusion coefficient of water is the diffusion coefficient of a single water molecule diffusing through bulk water. Mills determined the self-diffusion coefficient of water to be 2.299 $\times$ 10$^{-9}$ m$^2$/s at 298 K (see the table below for the variation of this value with temperature). In terms of other solvents, water’s diffusion is similar to toluene (2.267 $\times$ 10$^{-9}$ m$^2$/s) and higher than cyclohexane (1.1470 $\times$ 10$^{-9}$ m$^2$/s). Water’s diffusion is typical of small molecules and is actually one of the few properties for which it is ***not*** unusual.

**Experimental Self-Diffusion Coefficients for Water**
| Temperature ($^\circ$ C) | D ( $\times$ 10$^{-9}$ m$^2$/s) 
| --- | --- | 
|1|1.149|
|4|1.276|
|5|1.313|
|15|1.777|
|25|2.299|
|35|2.919|
|45|3.575|


As we have seen in the lectures for this section of the course, the most common way to calculate D is to use Einstein's relation over long simulation times,

$$D = \frac{1}{6}\frac{\left\langle \left| \mathbf{r}_{i}(t) - \mathbf{r}_{i}(0) \right|^{2}\right\rangle}{t} $$

where $\mathbf{r}_{i}(t)$ is the position of the $i^{\text{th}}$ particle at time $t$. The quantity $\left| \mathbf{r}_{i}(t) - \mathbf{r}_{i}(0) \right|$ is the displacement that particle $i$ has moved in time $t$. The angle brackets $\left\langle \right\rangle$ indicate a time-averaged property. 

If we consider the equation above carefully, we can see that by plotting the mean-squared displacement $\left\langle \left| \mathbf{r}_{i}(t) - \mathbf{r}_{i}(0) \right|^{2}\right\rangle $ as a function of time (i.e. $\left\langle \left| \mathbf{r}_{i}(t) - \mathbf{r}_{i}(0) \right|^{2}\right\rangle $ on the vertical axis, time $t$ on the horizontal axis), the diffusion coefficient $D$ can be calculated simply as **1/6 $\times$ the slope of this curve.**

Let's take our MD trajectory and use the ```MDAnalysis``` python module to calculate the self-diffusion coefficient of water using the TIP3P and TIP4P-Ew water modles. Just as for the RDF analysis above, ```MDAnalysis``` has a module specifically for calculating diffusion coefficients via the mean squared displacement method, described above. To start with, let's import this module:

In [ ]:
import MDAnalysis.analysis.msd as msd

And now we will tell ```MDAnalysis``` to measure the mean squared displacement of each individual water molecule in the simulation. Ideally we should use the centre of mass to define the 'position' of each water molecule. But for this workshop it is enough to approximate this position to be the position of the Oxygen atom (this position is quite close to the centre of mass anyway). In other words, we'll use the oxygen atom as a proxy for the centre of mass, and just measure the displacement of this atom instead. 

We have already defined our ```MDAnalysis``` universe above for our TIP3P simulation (```tip3p_u```). The code below defines a new object ```MSD``` which controls the parameters of the mean squared displacement calculation: 

In [ ]:
MSD = msd.EinsteinMSD(tip3p_u, select='name O', msd_type='xyz', fft=True)

Now we can run the analysis. We'll store the output in the variable ```tip3p_msd```: 

In [ ]:
MSD.run()

What does the plot of the mean-squared displacement $\left\langle \left| \mathbf{r}_{i}(t) - \mathbf{r}_{i}(0) \right|^{2}\right\rangle $ vs time look like? We can create this plot using the code below: 

In [ ]:
nframes = MSD.n_frames
timestep = 0.001*10 # this is the amount of real-time between MD frames in the DSD file you saved above. The time step is 0.001 ps, and the simulation is saved every 10 steps.
lagtimes = np.arange(nframes)*timestep # this is the array of times t, i.e. the horizontal axis values
tip3p_msd =  MSD.results.timeseries # this is the array of MSD values, i.e. the vertical axis values
plt.plot(lagtimes,tip3p_msd,label="TIP3P")
plt.legend()
plt.show()

We can now use this plot to determine the value of the self-diffusion coefficient $D$. Recalling from above, $D$ is 1/6 $\times$ the slope of this curve. To obtain the slope, we will use a new Python module, ```SciPy```, which includes a linear regression (i.e. 'line-of-best-fit) module called ```linregress```. To start with, let's load that into our notebook:

In [ ]:
from scipy.stats import linregress

For the simulation performed above, we can see that the correlation between the MSD variable and $t$ is linear roughly between 40 and 80 ps. Let's define these times as the start and end times for our linear regression analysis:

In [ ]:
start_time = 40
end_time = 80
start_index = int(start_time/timestep)
end_index = int(end_time/timestep)

We can let ```SciPy```'s ```linregress``` module do the rest, and we will save the output of the fitting into the new object ```linear_model```.

In [ ]:
linear_model = linregress(lagtimes[start_index:end_index],tip3p_msd[start_index:end_index])

The ```linear_model``` object contains two variables that we want - ```slope``` and ```stderr```. Let's define new variables that save this information: 

In [ ]:
slope = linear_model.slope
error = linear_model.stderr

We can now calculate the self-diffusion coefficient using Einstein's equation, above (making sure that we get the value in the correct units): 

In [ ]:
D = slope * 1/6*(1e-20)/1e-12*meter*meter/second
pm_D = error * 1/6*(1e-20)/1e-12*meter*meter/second
print("Self diffusion coefficient is",D,"+/-",pm_D)

#### Exercise ####

1. Use the code provided above to repeat the MSD analysis for the TIP4Pew water model MD simulations that you ran above. 
1. Use the code provided above to produce a single plot that compares the TIP3P and TIP4Pew water models' predictions for the self-diffusion coefficient in bulk water at 298 K. 
1. Compare your simulated water structure with the experimental data provided above. 

##### Important Notes #####
1. Insert as many code cells in the jupyter notebook below as you need. Try and keep your code as 'clean' as possible - it will help you solve and debug problems if and when they happen!
1. If you get stuck, ask your demonstrator for help!
1. <span style="color:red;">REALLY IMPORTANT - make sure that you change all "tip3p" to "tip4pew" in your new code below. Otherwise you will destroy the TIP3P RDFs you just calculated!</span>
1. <span style="color:red;">NOTE - the code above calculated the diffusion coefficient between 40-80 ps. These ranges are appropriate based on the data above, but YOUR simulation may require you to specify different ranges in the code below.</span>

## Discussion Questions and Information for Preparing Your Report ##